In [1]:
#NTRU

import numpy as np
import BasicNumTest as bnt

class RingElement:
    def __init__(self,num,mod): #Prime Field
        self.num = num % mod
        self.mod = mod
#        if bnt.MR_PT(prime) == False:
#            raise ValueError('The second input {} is not a prime'.format(prime))
        if self.num >= self.mod or self.num < 0:
            raise ValueError('The first input {} is not in the range 0 to {}'.format(num,prime-1))
        
    def __eq__(self, other):
        if other is None:
            return False
        return self.num == other.num and self.mod == other.mod
        
        
    def __add__(self, other):
        if self.mod != other.mod:
            raise TypeError('We cannot add two numbers in different rings.')
        num = (self.num + other.num) % self.mod
        return self.__class__(num, self.mod)
    
    def __sub__(self, other):
        if self.mod != other.mod:
            raise TypeError('We cannot subtract two numbers in different rings.')
        num = (self.num - other.num) % self.mod
        return self.__class__(num, self.mod)
    
    def __mul__(self, other):
        if self.mod != other.mod:
            raise TypeError('We cannot multiply two numbers in different rings.')
        num = (self.num * other.num) % self.mod
        return self.__class__(num, self.mod)
        
    def __pow__(self, exponent):
        a = exponent % (self.mod - 1)
        num = bnt.Exp(self.num, a, self.mod)
        return self.__class__(num, self.mod)
    
    def __neg__(self):
        num = self.mod - self.mod
        return self.__class__(num, self.mod)
    
    def __rmul__(self, coeff):
        num = (coeff * self.num) % self.mod
        return self.__class__(num, self.mod)
    
    def __truediv__(self, other):
        num = self.num * bnt.Inv(other.num,self.mod)
        
        return self.__class__(num, self.mod)
    
    
    
class PolyRing: ## x^N -1  / Quotient space. N은 항상 동일
    def __init__(self,N,coeffs,mod):
        self.N = N
        self.mod = mod
        self.coeffs = coeffs # Are coefficients ring element? integer?
        for i in range(self.N):
            if self.coeffs[i].mod != self.mod:
                raise ValueError('The second input {} is not a list of same ring elements'.format(coeffs))
        if len(coeffs) != N:
            raise ValueError('error')
#         if coeffs[N-1].num == 0:
#             raise ValueError('error')
                
    def __eq__(self,other):
        return self.N == other.N and self.coeffs == other.coeffs and self.mod == other.mod
    
    def __add__(self,other):
        if self.mod != other.mod:
            raise TypeError('We cannot add two polynomials in different rings.')
        c = list()
        
        for i in range(other.N):
            c.append(self.coeffs[i]+other.coeffs[i])

        return self.__class__(self.N,c,self.mod)
        
    def __sub__(self,other):
        if self.mod != other.mod:
            raise TypeError('We cannot add two polynomials in different rings.')
        c = list()
        for i in range(other.N):
            c.append(self.coeffs[i]-other.coeffs[i])
        return self.__class__(self.N,c,self.mod)
        
    def __mul__(self,other):
        if self.mod != other.mod:
            raise TypeError('We cannot add two polynomials in different rings.')
        c = list()
        for i in range(self.N): # output polynomial's coefficient
            temp = RingElement(0,self.mod)
            for j in range (other.N):
                temp = temp + (self.coeffs[j]*other.coeffs[(i-j)%self.N])
            c.append(temp)
        return self.__class__(self.N,c,self.mod)
    
    def __rmul__(self,coeff):
        c = []
        temp = RingElement(0,self.mod)
        for i in range(self.N):
            temp = coeff * self.coeffs[i]
            c.append(temp)
        return self.__class__(self.N,c,self.mod)
            

    def deg(self):
        degree = self.N
        temp = 0
        while temp == 0:
            degree -= 1
            if degree <0:
                return 0
            temp = self.coeffs[degree].num
        return degree
    
    def poly_div(self, other):
        if self.deg() >= other.deg():
            mx_coeffs = self.coeffs
            mx_deg = self.deg()
            mn_coeffs = other.coeffs
            mn_deg = other.deg()
        else:
            mx_coeffs = other.coeffs
            mx_deg = other.deg()
            mn_coeffs = self.coeffs
            mn_deg = self.deg()
                        
        
        q = self.N*[RingElement(0,self.mod)]
        r = mx_coeffs
        
        for i in range(mx_deg - mn_deg+1):
            if mx_coeffs[mx_deg-i].num!=0:
#                 d = mx_coeffs[mx_deg-i].num/mn_coeffs[mn_deg].num
                d = mx_coeffs[mx_deg-i]/mn_coeffs[mn_deg]
#                 q[mn_deg-i] = RingElement(d,self.mod)
                q[mn_deg-i] = d
    #             mx_coeffs[mx_deg-mn_deg-i:mx_deg-i] -= d * mn_coeffs
                for k in range(mn_deg+1):
                    mx_coeffs[mx_deg-mn_deg-i+k] -= d * mn_coeffs[k]

        mn = self.__class__(self.N,mn_coeffs,self.mod)
        quotient = self.__class__(self.N,q,self.mod)
        remainder = self.__class__(self.N,r,self.mod)
        
        return mn, quotient, remainder

    def Euclidean(self,other):
        a = self
        b = other
        while b.coeffs != self.N * [RingElement(0,self.mod)]:
            a, q, b = a.poly_div(b)
            for i in range(self.N):
                print(a.coeffs[i].num, q.coeffs[i].num, b.coeffs[i].num)
        return a
    
    def ExtendedEuclidean(self,other):
        a = self
        b = other
        one_coeffs = [RingElement(1,self.mod)] + (self.N-1) * [RingElement(0,self.mod)]
        one = self.__class__(self.N,one_coeffs,self.mod)
        zero_coeffs = self.N * [RingElement(0,self.mod)]
        zero = self.__class__(self.N,zero_coeffs,self.mod)
        M = [[one,zero],[zero,one]]
        while b.coeffs != self.N * [RingElement(0,self.mod)]:
            a,q,b = a.poly_div(b)
            M = [[M[1][0], M[1][1]],[M[0][0]-q*M[1][0], M[0][1]-q*M[1][1]]]
        return a, M[0][0], M[0][1]
        
#     def invpoly(self,other):
#         gcd, _, coeff = ExtendedEuclidean()
        
        

In [3]:
a0 = RingElement(0,7)
a1 = RingElement(1,7)
a2 = RingElement(2,7)
a3 = RingElement(3,7)
a4 = RingElement(4,7)

f = PolyRing(5,[a4,a4,a3,a4,a3],7)
g = PolyRing(5,[a1,a2,a1,a0,a0],7)

A,qq,rr = f.poly_div(g)

for i in range(A.N):
    print(A.coeffs[i].num, rr.coeffs[i].num, qq.coeffs[i].num)

for i in range(f.N):
    print(f.coeffs[i].num)
    
# B = A*qq + rr

# for i in range(B.N):
#     print(B.coeffs[i].num)

1 0 4
2 5 5
1 0 3
0 0 0
0 0 0
0
5
0
0
0


In [6]:
f = PolyRing(5,[a4,a4,a3,a4,a3],7)
g = PolyRing(5,[a1,a2,a1,a0,a0],7)

f-g

# dd,mm = f.ExtendedEuclidean(g)

# for i in range(C.N):
#     print(C.coeffs[i].num)

In [7]:
a7 = RingElement(7,7)
f = PolyRing(5,[a4,a4,a3,a4,a3],7)
g = PolyRing(5,[a1,a7,a7,a7,a7],7)

C = f.Euclidean(g)

for i in range(C.N):
    print(C.coeffs[i].num)

1 3 0
0 4 0
0 4 0
0 3 0
0 4 0
1
0
0
0
0


In [8]:
M = [[f,g], [g,f]]

M[1][1] == f

True

In [9]:
M = [[1,2],[3,4]]
M = [[M[1][1],M[1][0]+M[1][1]],[M[0][0],M[1][0]]]
M

[[4, 7], [1, 3]]